# Adversarial AttacksAnd Interpretability Covid Chestxray Dataset

Final Project - Cognitive Computing and Artificial Intelligence - UniCT a.a 2019/2020

*   Raiti Mario O55000434
*   Sortino Renato O55000405
*   Nardo Gabriele Salvatore O55000430

## Import Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os, random, torch, time, copy
import torch.nn.functional as F
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset
from PIL import Image, ImageOps, ImageFilter
from skimage.filters import threshold_local
from torch.autograd import Variable

## Enable Cuda for GPU computing

In [ ]:
# Setup device
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(dev)

## Create paths

In [ ]:
# dataset_path = './dataset'
from os import path
from google.colab import drive
drive.mount("/content/drive")
#covid_dataset_path = "/content/drive/My Drive/Covid_Chestray_Dataset"
covid_dataset_path = "/content/drive/.shortcut-targets-by-id/1jrhnTdvTCvooz8T_SbI3qAsDLr08pKrk/Covid_Chestray_Dataset/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Creating a Dataset

In [ ]:
classes = ['no-covid', 'covid']

In [ ]:
class image_dataset(Dataset):
    """Class creator for the x-ray dataset."""

    def __init__(self, csv_path, root_dir, transform=None, phase=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = pd.read_csv(csv_path)
        self.root_dir = root_dir
        self.transform = transform
        # If not a PA view, drop the line 
        self.df.drop(self.df[self.df.view != 'PA'].index, inplace=True)
        self.phase = phase

    def __len__(self):
        
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        if self.df['finding'].iloc[idx] != 'COVID-19':
            finding = 0
            img_path = os.path.sep.join([covid_dataset_path, 'images', self.df['filename'].iloc[idx]])
            image = Image.open(img_path)
            sample = {'image': image, 'finding': finding}
            
            if self.transform:
                sample = {'image': self.transform[self.phase](sample['image']), 'finding': finding}

        else:
            finding = 1
            img_path = os.path.sep.join([covid_dataset_path, 'images', self.df['filename'].iloc[idx]])
            image = Image.open(img_path)
            sample = {'image': image, 'finding': finding}

            if self.transform:
                sample = {'image': self.transform[self.phase](sample['image']), 'finding': finding}

        return sample

In [ ]:
xray_dataset = image_dataset(csv_path=os.path.sep.join([covid_dataset_path, 'metadata.csv']), root_dir=covid_dataset_path)

## Creating the data transformations

In [ ]:
class HistEqualization(object):
    """Image pre-processing.

    Equalize the image historgram
    """
    
    def __call__(self,image):
        
        return ImageOps.equalize(image, mask = None) 

In [ ]:
class ContrastBrightness(object):
    """Image pre-processing.

    alpha = 1.0 # Simple contrast control [1.0-3.0]
    beta = 0    # Simple brightness control [0-100]
    """
    
    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta
    
    def __call__(self,image,):
        image = np.array(image)
        for y in range(image.shape[0]):
            for x in range(image.shape[1]):
                image[y,x] = np.clip(self.alpha*image[y,x] + self.beta, 0, 255)

                return Image.fromarray(np.uint8(image)*255)

In [ ]:
class SmoothImage(object):
    """Image pre-processing.

    Smooth the image
    """
    def __call__(self,image):
        
        return image.filter(ImageFilter.SMOOTH_MORE)

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(1),
        transforms.RandomRotation(30, fill=(0,)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        ContrastBrightness(1.2,25),
        HistEqualization(),
        SmoothImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.5],
                             [0.25])
    ]),
    'test': transforms.Compose([
        transforms.Grayscale(1),
        transforms.Resize(240),
        transforms.CenterCrop(224),
        ContrastBrightness(1.2,25),
        HistEqualization(),
        SmoothImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.5],
                             [0.25])
    ]),
}

## Create a train, validation and test dataset and the dataloaders

In [ ]:
from torch.utils.data import DataLoader, Subset
# Import images as Dataset
image_datasets = {
    x: image_dataset(
        csv_path=os.path.sep.join([covid_dataset_path, 'metadata.csv']),
        root_dir=covid_dataset_path,
        transform=data_transforms,
        phase=x)
    for x in ['train', 'test']
}

46

In [ ]:
"""Ratio Train -> 70%, Val -> 15%, Test -> 15%"""
total_len = len(image_datasets['train'])

# Fractions
train_frac = 0.7
val_frac = 0.15
test_frac = 0.15
# Compute number of samples
num_train = int(total_len * train_frac)
num_val = int(total_len * val_frac)
num_test = int(total_len * test_frac)
# Indexes 
indexes = list(range(total_len))
# Split training set
train_idx = indexes[:num_train]
val_idx = indexes[num_train : num_train + num_val]
test_idx = indexes[num_train + num_val:]

In [ ]:
#Create train , val and test dataset and loaders as subset of image dataset
train_dataset = Subset(image_datasets['train'], train_idx)
val_dataset = Subset(image_datasets['test'], val_idx)
test_dataset = Subset(image_datasets['test'], test_idx)

train_loader = DataLoader(train_dataset, batch_size=64, num_workers=4, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=64, num_workers=4, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=64, num_workers=4, shuffle=False)

loaders = {"train": train_loader,
           "val": val_loader,
           "test": test_loader}

dataset_sizes = {x: len(loaders[x].dataset) for x in ['train', 'val', 'test']}

## Create a train method

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, device='cpu'):
    """
    Support function for model training.

    Args:
      model: Model to be trained
      criterion: Optimization criterion (loss)
      optimizer: Optimizer to use for training
      scheduler: Instance of ``torch.optim.lr_scheduler``
      num_epochs: Number of epochs
      device: Device to run the training on. Must be 'cpu' or 'cuda'
    """
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    try:
      for epoch in range(num_epochs):
          print('Epoch {}/{}'.format(epoch, num_epochs - 1))
          print('-' * 10)

          # Each epoch has a training and validation phase
          for phase in ['train', 'val', 'test']:
              if phase == 'train':
                  model.train()  # Set model to training mode
              else:
                  model.eval()   # Set model to evaluate mode

              running_loss = 0.0
              running_corrects = 0

              # Iterate over data.
              for data in loaders[phase]:
                  inputs = data['image']
                  labels = data['finding']
                  inputs = inputs.to(device)
                  labels = labels.to(device)
                  # zero the parameter gradients
                  optimizer.zero_grad()
                  # forward
                  # track history if only in train
                  with torch.set_grad_enabled(phase == 'train'):
                      outputs = model(inputs)
                      _, preds = torch.max(outputs, 1)
                      loss = criterion(outputs, labels)
                      # backward + optimize only if in training phase
                      if phase == 'train':
                          loss.backward()
                          optimizer.step()
                  # statistics
                  running_loss += loss.item() * inputs.size(0)
                  running_corrects += torch.sum(preds == labels.data)
              if phase == 'train':
                  scheduler.step()

              epoch_loss = running_loss / dataset_sizes[phase]
              epoch_acc = running_corrects.double() / dataset_sizes[phase]
              print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                  phase, epoch_loss, epoch_acc))
              # deep copy the model
              if phase == 'test' and epoch_acc > best_acc:
                  best_acc = epoch_acc
                  best_model_wts = copy.deepcopy(model.state_dict())
    except KeyboardInterrupt:
      time_elapsed = time.time() - since
      print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
      print('Best validation Acc: {:4f}'.format(best_acc))
      # load best model weights
      model.load_state_dict(best_model_wts)
      return model

## CNNs for classification

### Case 1 : Pre-trained model fine tuned "resnet18"
### Case 2 : From scratch model

## Case 1

## Load the pre-trained model

In [ ]:
model = models.resnet18(
    pretrained=True,
    progress=True
)

## Add a front layer to receive the black and white images from the X-Ray

In [ ]:
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

## Add a classification layer

In [ ]:
num_ftrs = model.fc.in_features

model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(num_ftrs, num_ftrs),
    nn.Dropout(p=0.5),
    nn.Linear(num_ftrs, 2),
)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = True

In [ ]:
criterion = nn.CrossEntropyLoss()

# Note that we are only training the head.
optimizer_ft = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.1)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.3)

## Fine tuning the model

In [ ]:
model_ft = train_model(model.to(device), criterion, optimizer_ft, exp_lr_scheduler,
                        num_epochs=50, device=device)

## Case 2

## Evaluation

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in dataloaders['test']:
        inputs = data['image']
        labels = data['finding']
        outputs = model_ft(inputs.float().to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()

print('Accuracy of the network: %d %%' % (
    100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))


with torch.no_grad():
    for data in dataloaders['test']:
        images = data['image']
        labels = data['finding']
        outputs = model_ft(images.to(device))
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels.to(device)).squeeze()
        for i in range(images.shape[0]):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(2):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
dataloader = torch.utils.data.DataLoader(image_datasets['test'], batch_size=(len(image_datasets['test'])), num_workers=8)

dataiter = iter(dataloader)
data = dataiter.next()
images = data['image']
labels = data['finding']

model_ft.to('cpu')

output = torch.tensor(model_ft(images).detach().numpy())

In [ ]:
from sklearn.metrics import classification_report
# show a nicely formatted classification report
print(classification_report(labels, np.argmax(output,1), target_names=classes))

In [ ]:
from sklearn.metrics import confusion_matrix

# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity
cm = confusion_matrix(labels, np.argmax(output,1))
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
# show the confusion matrix, accuracy, sensitivity, and specificity
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

[[ 29  25]
 [  5 136]]
acc: 0.8462
sensitivity: 0.5370
specificity: 0.9645
